In [5]:
import pandas as pd

#lendo a base de dados
df_vendas = pd.read_excel(r"Bases de Dados/Vendas.xlsx")
display(df_vendas)

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,1,Sapato Estampa,1,358,358
1,1,2019-01-01,1,Camiseta,2,180,360
2,1,2019-01-01,1,Sapato Xadrez,1,368,368
3,2,2019-01-02,3,Relógio,3,200,600
4,2,2019-01-02,3,Chinelo Liso,1,71,71
...,...,...,...,...,...,...,...
100994,69996,2019-12-26,17,Short Listrado,2,102,204
100995,69996,2019-12-26,17,Mochila,4,270,1080
100996,69996,2019-12-26,17,Pulseira Estampa,1,87,87
100997,69997,2019-12-26,11,Camisa Listrado,1,108,108


In [6]:
#verificando a base de dados
df_vendas.info()

<class 'pandas.DataFrame'>
RangeIndex: 100999 entries, 0 to 100998
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Código Venda    100999 non-null  int64         
 1   Data            100999 non-null  datetime64[us]
 2   ID Loja         100999 non-null  int64         
 3   Produto         100999 non-null  str           
 4   Quantidade      100999 non-null  int64         
 5   Valor Unitário  100999 non-null  int64         
 6   Valor Final     100999 non-null  int64         
dtypes: datetime64[us](1), int64(5), str(1)
memory usage: 5.4 MB


In [ ]:
#separando o faturamento por loja diario/anaul
faturamento_dia = (
    df_vendas.groupby(["ID Loja", "Data"], as_index=False)["Valor Final"]
    .sum()
    .rename(columns={"Valor Final":"Faturamento Diario"})
    )

df_vendas["Ano"] = df_vendas["Data"].dt.year

faturamento_ano = (
    df_vendas.groupby(["ID Loja", "Ano"], as_index=False)["Valor Final"]
    .sum()
    .rename(columns={"Valor Final":"Faturamento Anual"})
    )

display(faturamento_dia)
display(faturamento_ano.head())

,ID Loja,Data,Faturamento Diario
0,1,2019-01-01,1086
1,1,2019-01-02,8015
2,1,2019-01-03,5735
3,1,2019-01-04,9129
4,1,2019-01-05,5213
...,...,...,...
8938,25,2019-12-21,3737
8939,25,2019-12-22,9027
8940,25,2019-12-23,4355
8941,25,2019-12-24,1960


,ID Loja,Ano,Faturamento Anual
0,1,2019,1699681
1,2,2019,1590441
2,3,2019,1711968
3,4,2019,1674824
4,5,2019,1663770


In [11]:
#separando a variedade de itens vendidos por loja diaria/anual
variedade_itens_dia = (
    df_vendas.groupby(["ID Loja", "Data"], as_index=False)["Produto"]
    .nunique()
    .rename(columns={"Produto":"Variedade Diaria"})
)

variedade_itens_ano = (
    df_vendas.groupby(["ID Loja", "Ano"], as_index=False)["Produto"]
    .nunique()
    .rename(columns={"Produto":"Variedade Anual"})
)

display(variedade_itens_dia)
display(variedade_itens_ano.head())

,ID Loja,Data,Variedade Diaria
0,1,2019-01-01,3
1,1,2019-01-02,14
2,1,2019-01-03,12
3,1,2019-01-04,18
4,1,2019-01-05,6
...,...,...,...
8938,25,2019-12-21,9
8939,25,2019-12-22,19
8940,25,2019-12-23,6
8941,25,2019-12-24,6


,ID Loja,Ano,Variedade Anual
0,1,2019,120
1,2,2019,120
2,3,2019,120
3,4,2019,120
4,5,2019,120


In [18]:
#separando o ticket-medio de cada loja dia/ano
ticket_medio_dia = (
    df_vendas.groupby(["ID Loja", "Data"], as_index=False)[["Quantidade", "Valor Final"]]
    .sum()
    .assign(
        Ticket_Medio_Dia=lambda x: (
            x["Valor Final"] / x["Quantidade"].replace(0, pd.NA)
        ).round(2)
    )
    .drop(columns=["Valor Final", "Quantidade"])
    .rename(columns=lambda x: x.replace("_", " "))
)   

ticket_medio_ano = (    
    df_vendas.groupby(["ID Loja", "Ano"], as_index=False)[["Quantidade", "Valor Final"]]
    .sum()
    .assign(
        Ticket_Medio_Ano=lambda x: (
            x["Valor Final"] / x["Quantidade"].replace(0, pd.NA)
        ).round(2)
    )
    .drop(columns=["Valor Final", "Quantidade"])
    .rename(columns=lambda x: x.replace("_", " "))
)

display(ticket_medio_dia)
display(ticket_medio_ano)

,ID Loja,Data,Ticket Medio Dia
0,1,2019-01-01,271.50
1,1,2019-01-02,242.88
2,1,2019-01-03,204.82
3,1,2019-01-04,202.87
4,1,2019-01-05,372.36
...,...,...,...
8938,25,2019-12-21,155.71
8939,25,2019-12-22,265.50
8940,25,2019-12-23,290.33
8941,25,2019-12-24,163.33


,ID Loja,Ano,Ticket Medio Ano
0,1,2019,198.10
1,2,2019,193.81
2,3,2019,189.92
3,4,2019,194.09
4,5,2019,190.58
5,6,2019,189.11
6,7,2019,194.38
7,8,2019,189.32
8,9,2019,190.04
9,10,2019,188.28
